**IMPORTS**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

**LOAD FILES**

In [2]:
# Load the uploaded CSV files
ola_items_file_path = 'OLA_rd5500.csv'
ola_sales_file_path = 'OLA_rd5000.csv'

# Read the files into DataFrames
df_ola_items = pd.read_csv(ola_items_file_path)
df_ola_sales = pd.read_csv(ola_sales_file_path)

**EDA FOR RD5500 (STORE ITEMS)**

In [3]:
print("Before data cleansing:")
print(df_ola_items.count())

Before data cleansing:
INCODE      4104
ITE_DESC    4104
DEP_CODE    4104
UNT_PRIC    4104
UNIT           0
dtype: int64


In [4]:
df_ola_items.head()

,INCODE,ITE_DESC,DEP_CODE,UNT_PRIC,UNIT
0,SLD11,ROASTED SESAME DRESSING,9,0.0,NaN
1,SLD7,NENA'S HARVEST,9,0.0,NaN
2,REPSHK9,RIPE MANGO SHAKE,16,0.0,NaN
3,REPPST4,SPAGHETTI WITH PB,16,0.0,NaN
4,REPCHM9,SPAGHETTI WITH CHICKEN,16,0.0,NaN


In [5]:
print(df_ola_items.dtypes)

INCODE       object
ITE_DESC     object
DEP_CODE      int64
UNT_PRIC    float64
UNIT        float64
dtype: object


**CLEAN RD5500 DATA**

In [6]:
# Drop all columns except 'INCODE' and 'ITE_DESC'
df_ola_items = df_ola_items[['INCODE', 'ITE_DESC']]

# Get all unique pairs of 'INCODE' and 'ITE_DESC'
df_ola_filtered_items = df_ola_items.drop_duplicates()

In [7]:
print("After data cleansing:")
print(df_ola_filtered_items.count())

After data cleansing:
INCODE      856
ITE_DESC    856
dtype: int64


In [8]:
pip install xlwt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
output_file = 'store_items.csv'
df_ola_filtered_items.to_csv(output_file, index=False)

**MERGE SIMILAR STORE ITEMS BASED ON ITEM DESCRIPTION**

In [10]:
# Normalize the ITE_DESC column to make it case-insensitive and group similar items
df_ola_filtered_items.loc[:, 'Normalized_ITE_DESC'] = df_ola_filtered_items['ITE_DESC'].str.strip().str.lower()

# Group by the normalized item description and aggregate INCODE into a list
grouped_df = df_ola_filtered_items.groupby('Normalized_ITE_DESC').agg({
    'INCODE': list,
    'ITE_DESC': 'first'  # Take the first occurrence of the original case item description
}).reset_index()

# Rename columns for clarity
grouped_df = grouped_df.rename(columns={'INCODE': 'INCODE_LIST', 'ITE_DESC': 'ITEM_DESC'})

# Drop the normalized column as it is no longer needed
grouped_df = grouped_df[['INCODE_LIST', 'ITEM_DESC']]
grouped_df.head(20)

/tmp/ipykernel_4666/939108571.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ola_filtered_items.loc[:, 'Normalized_ITE_DESC'] = df_ola_filtered_items['ITE_DESC'].str.strip().str.lower()


,INCODE_LIST,ITEM_DESC
0,[BL67],1CAJCHX SPG PBRD CNC IT
1,[BULK7],1HALF SERVING SOUTHWEST
2,[BDS6],1P BIGG CAJUN CHICKEN
3,[BULK2],1PBHAM 4CHX 10BM SOUTHS 1BBACK
4,[BULK6],1PC BBYBACK
5,"[DEL8, CHM6, XTR2, REPCHM6]",1PC BIGG CAJUN CHICKEN
6,"[REPCHM4, CHM4, DEL3, XTR1, PP3, BDS11, REPXTR...",1PC BIGG CHICKEN
7,"[REPSID7, SID5, XTR15, BDS77]",1PC BMP
8,[MP130],1PC CHIX PJ
9,[DEL165],1PC CHIX IT


In [11]:
print("After incode merge:")
print(grouped_df.count())

After incode merge:
INCODE_LIST    342
ITEM_DESC      342
dtype: int64


In [12]:
output_file = 'ola_grouped_items.csv'
grouped_df.to_csv(output_file, index=False)

**EDA FOR RD5000 DATA**

In [13]:
df_ola_sales.head()

,BRANCH,POS,TRANSDATE,ITE_CODE,QUANTITY,DEP_CODE,DATE,TIME,TYPE,DELIVERY
0,OLA,1,2023-12-31,SLD11,1,9,2023-12-31,21:53,D,0
1,OLA,1,2023-12-31,SLD7,1,9,2023-12-31,21:53,D,0
2,OLA,1,2023-12-31,REPSHK9,1,16,2023-12-31,21:53,D,0
3,OLA,1,2023-12-31,REPPST4,1,16,2023-12-31,21:53,D,0
4,OLA,1,2023-12-31,REPCHM9,2,16,2023-12-31,21:53,D,0


In [14]:
print(df_ola_sales.dtypes)

BRANCH       object
POS           int64
TRANSDATE    object
ITE_CODE     object
QUANTITY      int64
DEP_CODE      int64
DATE         object
TIME         object
TYPE         object
DELIVERY      int64
dtype: object
